<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/imdb/Run_IMDB_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function
!rm -r abstention
!git clone https://github.com/blindauth/abstention.git
!pip uninstall abstention
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-plggbz8t/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 2940 (delta 0), reused 1 (delta 0), pack-reused 2938
Receiving objects: 100% (2940/2940), 264.92 MiB | 36.54 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [0]:
import gzip
import numpy as np

def read_vals(gzip_file, idx_to_load=0):
    return np.array([[
        float(y) for y in x.decode("utf-8").rstrip().split("\t")][idx_to_load]
                          for x in gzip.open(gzip_file)])
  
def get_calc_tpr_at_fpr_func(fpr_level): 
    def tpr_at_fpr(y_true, y_score):
      #verify probs are sorted, if not, sort them
      if (np.min(y_score[1:] - y_score[:-1]) < 0):
          (sorted_y_score, sorted_y_true) = zip(*sorted(zip(y_score,y_true),
                                                   key=lambda x: x[0]))
          sorted_y_score = np.array(sorted_y_score)
          sorted_y_true = np.array(sorted_y_true)
      else:
          sorted_probs = y_score
          sorted_y_true = y_true
      pos_cumsum = np.cumsum(sorted_y_true)
      tprs = (pos_cumsum[-1]-pos_cumsum)/pos_cumsum[-1]
      neg_cumsum = np.cumsum(1-sorted_y_true)
      fprs = (neg_cumsum[-1]-neg_cumsum)/neg_cumsum[-1]
      for (tpr,fpr) in zip(tprs,fprs):
          if (fpr <= fpr_level):
              return tpr    
    return tpr_at_fpr

In [3]:
import numpy as np
import random
from abstention.abstention import (
    MonteCarloWindowAbstDeltaTprAtFprThreshold,
    EstWindowAbstDeltaAuroc,
    Uncertainty, NegPosteriorDistanceFromThreshold,
    FixedThreshold, OneMinusJSDivFromClassFreq,
    DualThresholdsFromPointFiveOnValidSet)
from abstention.calibration import PlattScaling
from abstention.label_shift import EMImbalanceAdapter, NoAdaptation
from sklearn.metrics import roc_auc_score
from collections import OrderedDict
import sys
import json
import os


model_dirs = [
  "record_1_model_FObIR_preds",
  "record_2_model_T4HRC_preds",
  "record_3_model_ZGX50_preds",
  "record_4_model_hWBrJ_preds",
  "record_5_model_mBWKc_preds",
  "record_6_model_mgGta_preds",
  "record_7_model_Si0xy_preds",
  "record_8_model_yds3D_preds",
  "record_9_model_dCvxV_preds",
  "record_10_model_Vs2cs_preds"]

target_fpr_level = 0.01
n_samples=100
smoothing_window_size=11

orig_test_labels = read_vals("abstention_experiments/imdb/test_labels.txt.gz")
orig_valid_labels = read_vals("abstention_experiments/imdb/valid_labels.txt.gz")

bootstrap_test_set_size = len(orig_valid_labels) #bootstrapped to be this big


metricname_to_computefunc = OrderedDict([
    ('tpratfpr_'+str(target_fpr_level),
     get_calc_tpr_at_fpr_func(target_fpr_level)),
    ('auroc', roc_auc_score)
])

abstname_to_factory = OrderedDict([
    ('mcwindowdelta_tpratfpr_30pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.3*bootstrap_test_set_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('fumera_tpratfpr_30pc',
     DualThresholdsFromPointFiveOnValidSet(
         fracs_to_abstain_on=[0.3],
         metric=get_calc_tpr_at_fpr_func(target_fpr_level))),
    ('mcwindowdelta_tpratfpr_15pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.15*bootstrap_test_set_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('fumera_tpratfpr_15pc',
     DualThresholdsFromPointFiveOnValidSet(
         fracs_to_abstain_on=[0.15],
         metric=get_calc_tpr_at_fpr_func(target_fpr_level))),   
    ('js', OneMinusJSDivFromClassFreq()),
    ('estwindowdelta_auroc_30pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.3*bootstrap_test_set_size),
         return_max_across_windows=True)),
    ('estwindowdelta_auroc_15pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.15*bootstrap_test_set_size),
         return_max_across_windows=True)),
    ('fumera_auroc_30pc', DualThresholdsFromPointFiveOnValidSet(
                           fracs_to_abstain_on=[0.3],
                           metric=roc_auc_score)),
    ('fumera_auroc_15pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.15],
                          metric=roc_auc_score)),
    ('uncertainty', Uncertainty()),
    ('distfrom0.5', NegPosteriorDistanceFromThreshold(FixedThreshold(0.5))),  
])

calibfactory = PlattScaling(verbose=False)
adaptname_to_factory = OrderedDict([
    ('EM', EMImbalanceAdapter(verbose=False)),
    ('None', NoAdaptation())])

abstfracs = [0.15, 0.3]

num_trials_per_model = 3

###
#initialize everything
metricname_to_adaptname_to_baselineperfs = OrderedDict()
for metricname in metricname_to_computefunc:
  adaptname_to_baselineperfs = OrderedDict()
  for adaptname in adaptname_to_factory:
    adaptname_to_baselineperfs[adaptname] = []
  metricname_to_adaptname_to_baselineperfs[metricname] =(
    adaptname_to_baselineperfs)
  
metricname_to_abstfrac_to_abstnadapt_to_perf = OrderedDict()
for metricname in metricname_to_computefunc:
  abstfrac_to_abstnadapt_to_perf = OrderedDict()
  for abstfrac in abstfracs:
    abstnadapt_to_perf = OrderedDict()
    for abstname in abstname_to_factory:
      for adaptname in adaptname_to_factory:
        abstnadapt_to_perf[abstname+":"+adaptname] = []
    abstfrac_to_abstnadapt_to_perf[abstfrac] = abstnadapt_to_perf
  metricname_to_abstfrac_to_abstnadapt_to_perf[metricname] = (
    abstfrac_to_abstnadapt_to_perf)
  
####
for model_dir_idx,model_dir in list(enumerate(model_dirs))[:]:
  print("\n###On model dir",model_dir)
  orig_test_detpreds = read_vals(
      "abstention_experiments/imdb/"+model_dir+"/test_det_preds.txt.gz")
  orig_valid_detpreds = read_vals(
      "abstention_experiments/imdb/"+model_dir+"/valid_det_preds.txt.gz")
  orig_test_nondetpreds = np.array([
      read_vals("abstention_experiments/imdb/"
                +model_dir+"/test_nondet_preds_"+str(i)+".txt.gz")
      for i in range(100)]).transpose((1,0))
  
  for trial_idx in range(num_trials_per_model):
    
    print("\n###trial idx",trial_idx)
    
    rng = np.random.RandomState(trial_idx*100 + model_dir_idx*10)  
    bootstrap_valid_indices = rng.choice(
        a=list(range(len(orig_valid_labels))),
        size=len(orig_valid_labels), replace=True)
    indices_of_test_positives = [x[0] for x in enumerate(orig_test_labels)
                                 if x[1]==1]
    indices_of_test_negatives = [x[0] for x in enumerate(orig_test_labels)
                                 if x[1]==0]
    
    bootstrap_test_positive_indices = rng.choice(
        a=indices_of_test_positives,
        size=int(len(orig_valid_labels)/(3.0)),
        replace=True)
    bootstrap_test_negative_indices = rng.choice(
        a=indices_of_test_negatives,
        size=(len(orig_valid_labels)-len(bootstrap_test_positive_indices)),
        replace=True)
    bootstrap_test_indices =(list(bootstrap_test_negative_indices)+
                             list(bootstrap_test_positive_indices))
    
    bootstrap_valid_labels = orig_valid_labels[bootstrap_valid_indices]
    bootstrap_valid_detpreds = orig_valid_detpreds[bootstrap_valid_indices]
    bootstrap_test_labels = orig_test_labels[bootstrap_test_indices]
    bootstrap_test_detpreds = orig_test_detpreds[bootstrap_test_indices]
    bootstrap_test_nondetpreds = orig_test_nondetpreds[bootstrap_test_indices]
      
    #perform the calibration
    calib_func = calibfactory(valid_preacts=bootstrap_valid_detpreds,
                               valid_labels=bootstrap_valid_labels)
    calib_bootstrap_valid_detpreds = calib_func(
        preact=bootstrap_valid_detpreds)
    calib_bootstrap_test_detpreds = calib_func(
        preact=bootstrap_test_detpreds)
    calib_bootstrap_test_nondetpreds = (
      np.array([calib_func(x) for x in
                bootstrap_test_nondetpreds.transpose((1,0))
               ]).transpose((1,0)))

    adaptname_to_bootstrap_test_detpreds = OrderedDict()
    adaptname_to_bootstrap_test_nondetpreds = OrderedDict()
    #perform the adaptation
    for adaptname in adaptname_to_factory:  
      adaptation_func = adaptname_to_factory[adaptname](
          valid_labels=bootstrap_valid_labels,
          tofit_initial_posterior_probs=calib_bootstrap_test_detpreds,
          valid_posterior_probs=calib_bootstrap_valid_detpreds)
      adapted_bootstrap_test_detpreds = adaptation_func(
          unadapted_posterior_probs=calib_bootstrap_test_detpreds)
      adaptname_to_bootstrap_test_detpreds[adaptname] =\
        adapted_bootstrap_test_detpreds
      adapted_bootstrap_test_nondetpreds = np.array(
          [adaptation_func(x) for x in
           calib_bootstrap_test_nondetpreds.transpose((1,0))]).transpose((1,0))
      adaptname_to_bootstrap_test_nondetpreds[adaptname] =\
        adapted_bootstrap_test_nondetpreds
       
    for adaptname in adaptname_to_factory:
      #compute the baseline test perfs
      for metricname in metricname_to_computefunc:
        computefunc = metricname_to_computefunc[metricname]
        perf = computefunc(
            y_true=bootstrap_test_labels,
            y_score=adaptname_to_bootstrap_test_detpreds[adaptname])        
        metricname_to_adaptname_to_baselineperfs[
            metricname][adaptname].append(perf)
        print("Base perf",metricname,perf)
      #perform abstentions   
      for abstname in abstname_to_factory:
        print("\nOn method", abstname+":"+adaptname)
        abstention_func = abstname_to_factory[abstname](
            valid_labels=bootstrap_valid_labels,
            valid_posterior=calib_bootstrap_valid_detpreds)
        #get the abstention priorities, and sort the indices accordingly
        adapted_bootstrap_test_detpreds =\
          adaptname_to_bootstrap_test_detpreds[adaptname]
        var_adapted_bootstrap_test_nondetpreds = np.var(
            adaptname_to_bootstrap_test_nondetpreds[adaptname], axis=1)
        bootstrap_test_abstention_priorities = abstention_func(
            posterior_probs=adapted_bootstrap_test_detpreds,
            uncertainties=var_adapted_bootstrap_test_nondetpreds)
        sorted_indices = [
              x[0] for x in
              sorted(enumerate(bootstrap_test_abstention_priorities),
              key=lambda x: x[1])]

        for abstfrac in abstfracs:
          #get the post-abstention examples            
          num_to_retain = int(len(sorted_indices)*(1-abstfrac))
          retained_test_detpreds = adapted_bootstrap_test_detpreds[
              sorted_indices[:num_to_retain]]
          retained_test_labels =\
            bootstrap_test_labels[sorted_indices[:num_to_retain]]
          for metricname in metricname_to_computefunc:
            metric_compute_func = metricname_to_computefunc[metricname]
            perf = metric_compute_func(y_true=retained_test_labels,
                                       y_score=retained_test_detpreds)
            metricname_to_abstfrac_to_abstnadapt_to_perf[metricname]\
              [abstfrac][abstname+":"+adaptname].append(perf)
            print("Abstfrac",abstfrac,metricname,perf)


###On model dir record_1_model_FObIR_preds

###trial idx 0


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.38223822382238226
Base perf auroc 0.9363083150972964

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.6677577741407529
Abstfrac 0.15 auroc 0.9300288668054324
Abstfrac 0.3 tpratfpr_0.01 0.7598425196850394
Abstfrac 0.3 auroc 0.9548932444894352

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.502454991816694
Abstfrac 0.15 auroc 0.9159868520485597
Abstfrac 0.3 tpratfpr_0.01 0.6521739130434783
Abstfrac 0.3 auroc 0.9325606116270961

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5785876993166287
Abstfrac 0.15 auroc 0.938466524199798
Abstfrac 0.3 tpratfpr_0.01 0.7198795180722891
Abstfrac 0.3 auroc 0.9424421062246164

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5519366197183099
Abstfrac 0.15 auroc 0.9434949048820864
Abstfrac 0.3 tpratfpr_0.01 0.5666235446313066
Abstfrac 0.3 auroc 0.9443758983901638

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.4254790631653655
Abstfrac 0.15 auroc 0.954

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.3777377737773777
Base perf auroc 0.9351703881952618

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.6410256410256411
Abstfrac 0.15 auroc 0.9263313430073178
Abstfrac 0.3 tpratfpr_0.01 0.7709720372836218
Abstfrac 0.3 auroc 0.9543277812195597

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.45990180032733224
Abstfrac 0.15 auroc 0.9101042492883539
Abstfrac 0.3 tpratfpr_0.01 0.5720375106564365
Abstfrac 0.3 auroc 0.9256124771783643

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5559090909090909
Abstfrac 0.15 auroc 0.936533621933622
Abstfrac 0.3 tpratfpr_0.01 0.6783844427823486
Abstfrac 0.3 auroc 0.9428010372147617

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5160729800173762
Abstfrac 0.15 auroc 0.9435584865384956
Abstfrac 0.3 tpratfpr_0.01 0.49066002490660027
Abstfrac 0.3 auroc 0.9397935087298699

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.4142602495543672
Abstfrac 0.15 auroc 0.9

...

Abstfrac 0.15 tpratfpr_0.01 0.629569012547736
Abstfrac 0.15 auroc 0.9197960723895064
Abstfrac 0.3 tpratfpr_0.01 0.7368770764119601
Abstfrac 0.3 auroc 0.9474305545059084

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.46590289143480634
Abstfrac 0.15 auroc 0.9054571003037409
Abstfrac 0.3 tpratfpr_0.01 0.5991869918699188
Abstfrac 0.3 auroc 0.9232821856814756

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5246494798733605
Abstfrac 0.15 auroc 0.9318953304424265
Abstfrac 0.3 tpratfpr_0.01 0.6814159292035398
Abstfrac 0.3 auroc 0.9362477238469757

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4929275610801543
Abstfrac 0.15 auroc 0.9403511118002843
Abstfrac 0.3 tpratfpr_0.01 0.5107913669064749
Abstfrac 0.3 auroc 0.9509700057015692

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.3903724525650035
Abstfrac 0.15 auroc 0.9506267492389048
Abstfrac 0.3 tpratfpr_0.01 0.4380639868744873
Abstfrac 0.3 auroc 0.9636089441651421

On method estwindowd

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.36693669366936693
Base perf auroc 0.9268302111447082

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.6290234588106929
Abstfrac 0.15 auroc 0.9134344428441425
Abstfrac 0.3 tpratfpr_0.01 0.7392438070404173
Abstfrac 0.3 auroc 0.9442743359327854

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.46153846153846156
Abstfrac 0.15 auroc 0.8982859367669914
Abstfrac 0.3 tpratfpr_0.01 0.583732057416268
Abstfrac 0.3 auroc 0.9185592529245781

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5319338990620813
Abstfrac 0.15 auroc 0.9284061302665594
Abstfrac 0.3 tpratfpr_0.01 0.6686046511627907
Abstfrac 0.3 auroc 0.9317149922260082

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4873286248442044
Abstfrac 0.15 auroc 0.9398433135588247
Abstfrac 0.3 tpratfpr_0.01 0.4834801762114537
Abstfrac 0.3 auroc 0.941850147951672

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.4023834560112163
Abstfrac 0.15 auroc 0.94

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Base perf tpratfpr_0.01 0.3363336333633363
Base perf auroc 0.9259147753683423

On method mcwindowdelta_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.6115657392253137
Abstfrac 0.15 auroc 0.9138333590685441
Abstfrac 0.3 tpratfpr_0.01 0.6685678073510773
Abstfrac 0.3 auroc 0.947042724589629

On method fumera_tpratfpr_30pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.5073649754500819
Abstfrac 0.15 auroc 0.9028831700804486
Abstfrac 0.3 tpratfpr_0.01 0.6437546193643755
Abstfrac 0.3 auroc 0.9292062408847925

On method mcwindowdelta_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4917447568049978
Abstfrac 0.15 auroc 0.9277078490240454
Abstfrac 0.3 tpratfpr_0.01 0.6788732394366197
Abstfrac 0.3 auroc 0.935465571205008

On method fumera_tpratfpr_15pc:EM
Abstfrac 0.15 tpratfpr_0.01 0.4568676716917923
Abstfrac 0.15 auroc 0.9376986919764617
Abstfrac 0.3 tpratfpr_0.01 0.4701912260967379
Abstfrac 0.3 auroc 0.9443127414669631

On method js:EM
Abstfrac 0.15 tpratfpr_0.01 0.373510861948143
Abstfrac 0.15 auroc 0.94720

In [4]:
file_out = "imdb_abstention_results.json"
dict_to_write = {
    "metricname_to_adaptname_to_baselineperfs":
     metricname_to_adaptname_to_baselineperfs,
    "metricname_to_abstfrac_to_abstnadapt_to_perf":
     metricname_to_abstfrac_to_abstnadapt_to_perf}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0